# Get the data

The below link gets the faq documents in a json format. 

There is a script that was used to parse the faq from Google docs to Json using python
https://github.com/dimzachar/llm_zoomcamp/blob/master/notes/01-intro/retrieval-with-minsearch.md

this was covered in another workshop

In [1]:
# !wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

In [3]:
import json
import minSearch

In [ ]:
with open('documents.json','rt') as f_in:
    docs_raw = json.load(f_in)

In [8]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)
        
documents[:2] #list of dictioanry with all the document questions

# STRUCTURE 
# text: answer to the question
# section: section question belongs to
# question: actual question
# course: what course question belongs to

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp'}]

In [15]:
import pandas as pd
docs_df = pd.DataFrame(documents)
docs_df.groupby('course').count()

,text,section,question
course,,,
data-engineering-zoomcamp,435,435,435
machine-learning-zoomcamp,375,375,375
mlops-zoomcamp,138,138,138


## Indexing documents using minSearch

In [19]:
index = minSearch.Index(
    text_fields=['question', 'text', 'section'], #similarity matching, processed for TF-IDF vectorization
    keyword_fields=['course'] # exact matching
)

In [20]:
q = 'the course has already started, can i still join?'

In [21]:
index.fit(documents)

In [27]:
boost = {'question': 3.0, 'section':0.5} # when we know one field is important than other. like course

results = index.search(
    query=q,
    boost_dict=boost,
    filter_dict={'course': 'data-engineering-zoomcamp'}, 
    num_results=5
)
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineerin

## Generating answer with Claude

In [31]:
import anthropic
from dotenv import load_dotenv
import os

# Load environment variables from .env file into script environment
load_dotenv()


True

In [32]:
q

'the course has already started, can i still join?'

In [34]:
client = anthropic.Anthropic()

message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": q}
    ]
)
print(message)

BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Claude API. Please go to Plans & Billing to upgrade or purchase credits.'}}

In [35]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results


In [36]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT: 
    {context}
    """.strip()

    context = ""
    for doc in search_results:
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [42]:
def llm(prompt):
    from openai import OpenAI
    client = OpenAI(api_key = os.getenv('OPENAI_API_KEY'))
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [38]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [43]:
rag('the course has already started, can I still enroll?')


APIConnectionError: Connection error.

# Spin up an ElasticSearch container


In [8]:
# docker run -it \
#     --rm \
#     --name elasticsearch \
#     -m 4GB \
#     -p 9200:9200 \
#     -p 9300:9300 \
#     -e "discovery.type=single-node" \
#     -e "xpack.security.enabled=false" \
#     docker.elastic.co/elasticsearch/elasticsearch:8.4.3


# Ingest the document



In [3]:
from elasticsearch import Elasticsearch
from tqdm

SyntaxError: invalid syntax (2139028749.py, line 2)